In [31]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [32]:
data=pd.read_csv('cleaned_file.csv')
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3348,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
3349,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
3350,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
3351,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [33]:
data.head
data.info
data.isnull().sum

<bound method DataFrame.sum of         age    sex     cp  trestbps   chol    fbs  restecg  thalach  exang  \
0     False  False  False     False  False  False    False    False  False   
1     False  False  False     False  False  False    False    False  False   
2     False  False  False     False  False  False    False    False  False   
3     False  False  False     False  False  False    False    False  False   
4     False  False  False     False  False  False    False    False  False   
...     ...    ...    ...       ...    ...    ...      ...      ...    ...   
3348  False  False  False     False  False  False    False    False  False   
3349  False  False  False     False  False  False    False    False  False   
3350  False  False  False     False  False  False    False    False  False   
3351  False  False  False     False  False  False    False    False  False   
3352  False  False  False     False  False  False    False    False  False   

      oldpeak  slope     ca   th

In [34]:
data.describe
data["target"].describe
data["target"].isnull

<bound method Series.isnull of 0       0
1       0
2       0
3       0
4       0
       ..
3348    1
3349    0
3350    0
3351    1
3352    0
Name: target, Length: 3353, dtype: int64>

In [35]:
data["target"].isnull().sum()


np.int64(0)

In [36]:
data['target'].value_counts()
X = data.drop(columns=['target'])  
y = data['target']  

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((2347, 13), (1006, 13), (2347,), (1006,))

In [38]:
sc = StandardScaler()
x_train = sc.fit_transform(X_train)
x_test = sc.transform(X_test)

In [39]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train, Y_train)
Y_pred_xgb = xgb_model.predict(X_test)

In [40]:
Y_pred_xgb.shape

(1006,)

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

# **1. Linear Regression (Regression Model)**
lr_model = LinearRegression()
lr_model.fit(X_train, Y_train)
Y_pred_lr = lr_model.predict(X_test)
mse = mean_squared_error(Y_test, Y_pred_lr)
print(f"Linear Regression MSE: {mse}")

# Convert predictions to nearest integer class
Y_pred_rounded = Y_pred_lr.round()

# Compute Accuracy
lr_accuracy = accuracy_score(Y_test, Y_pred_rounded)
print(f"Linear Regression Accuracy: {lr_accuracy}")
# **2. Random Forest Classifier**

rf_model = RandomForestClassifier(n_estimators=20, max_depth=4, random_state=42)
rf_model.fit(X_train, Y_train)

# rf_model = RandomForestClassifier()
# rf_model.fit(X_train, Y_train)
Y_pred_rf = rf_model.predict(X_test)
rf_accuracy = accuracy_score(Y_test, Y_pred_rf)
print(f"Random Forest Accuracy: {rf_accuracy}")

# **3. Support Vector Classifier (SVC)**
svc_model = SVC()
svc_model.fit(X_train, Y_train)
Y_pred_svc = svc_model.predict(X_test)
svc_accuracy = accuracy_score(Y_test, Y_pred_svc)
print(f"SVM Accuracy: {svc_accuracy}")

# **4. Decision Tree Classifier**
dt_model = DecisionTreeClassifier(max_depth=4, random_state=42)
dt_model.fit(X_train, Y_train)
Y_pred_dt = dt_model.predict(X_test)
dt_accuracy = accuracy_score(Y_test, Y_pred_dt)
print(f"Decision Tree Accuracy: {dt_accuracy}")

# **5. k-Nearest Neighbors (KNN)**
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, Y_train)
Y_pred_knn = knn_model.predict(X_test)
knn_accuracy = accuracy_score(Y_test, Y_pred_knn)
print(f"KNN Accuracy: {knn_accuracy}")

from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model.fit(X_train, Y_train)
Y_pred_log = log_model.predict(X_test)
log_accuracy = accuracy_score(Y_test, Y_pred_log)
print(f"Logistic Regression Accuracy: {log_accuracy}")


Linear Regression MSE: 0.12541554161908572
Linear Regression Accuracy: 0.8349900596421471
Random Forest Accuracy: 0.8777335984095428
SVM Accuracy: 0.68389662027833
Decision Tree Accuracy: 0.8409542743538767
KNN Accuracy: 0.9055666003976143
Logistic Regression Accuracy: 0.8240556660039762


/usr/local/python/3.12.1/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [49]:
score_xgb = round(accuracy_score(Y_pred_xgb,Y_test)*100,2)

print("The accuracy score achieved using XGBoost is: "+str(score_xgb)+" %")

The accuracy score achieved using XGBoost is: 98.41 %


In [50]:
from keras.models import Sequential
from keras.layers import Dense

2025-04-09 09:53:07.800896: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-09 09:53:08.059948: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-09 09:53:08.210208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744192388.519208   21051 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744192388.609982   21051 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 09:53:09.253394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [51]:
sc = StandardScaler()
x_train = sc.fit_transform(X_train)
x_test = sc.transform(X_test)

In [52]:
model = Sequential()
model.add(Dense(11,activation='relu',input_dim=13))
model.add(Dense(11,activation='relu'))
model.add(Dense(11,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-09 09:53:53.497670: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [53]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
hist = model.fit(x_train, Y_train, batch_size=10, epochs=3)

Epoch 1/3
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6501 - loss: 0.6443
Epoch 2/3
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8276 - loss: 0.4010
Epoch 3/3
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8635 - loss: 0.3380


In [55]:
loss,accuracy = model.evaluate(x_test,Y_test)
print("LOSS:  " + "%.2f" % loss)
print("ACCURACY:  " + "%.2f" % accuracy)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8716 - loss: 0.3187  
LOSS:  0.33
ACCURACY:  0.87


In [57]:
import pickle
with open("finalxgb_model.pkl", "wb") as model_file:
    pickle.dump(xgb_model, model_file)

# Save the StandardScaler as well
with open("finalscaler.pkl", "wb") as scaler_file:
    pickle.dump(sc, scaler_file)